In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606957722427_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from datetime import date
today = date.today()

day = today.strftime("%d-%m-%Y")

datos = spark.read.csv('s3://proyecto-bigdata/raw/datos-{0}.csv'.format(day), inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
datos.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+------------+----------------+----------------+--------------------+----+-------------+----+--------------------+---------+------+----------------+----------------+----------+---------------------+------------+-----------------+-----------------+-----------------+--------+----------+
|fecha_reporte_web|id_de_caso|fecha_de_notificaci_n|departamento|departamento_nom|ciudad_municipio|ciudad_municipio_nom|edad|unidad_medida|sexo|fuente_tipo_contagio|ubicacion|estado|pais_viajo_1_cod|pais_viajo_1_nom|recuperado|fecha_inicio_sintomas|fecha_muerte|fecha_diagnostico| fecha_recuperado|tipo_recuperacion|per_etn_|nom_grupo_|
+-----------------+----------+---------------------+------------+----------------+----------------+--------------------+----+-------------+----+--------------------+---------+------+----------------+----------------+----------+---------------------+------------+-----------------+-----------------+-----------------+--------+-

In [4]:
datos_limpios = datos.drop('ciudad_municipio','departamento','unidad_medida','pais_viajo_1_cod','per_etn_','nom_grupo_')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def edad_etapa(edad):
    etapa = ''
    if (edad<=5):
        etapa = 'primera infancia'
    elif (5<edad<=11):
        etapa = 'infancia'
    elif (11<edad<=18):
        etapa = 'adolescencia'
    elif (18<edad<=26):
        etapa = 'juventud'
    elif (26<edad<=59):
        etapa = 'adultez'
    elif (edad>=60):
        etapa = 'vejez'
    return etapa

#Agregar Columna
etapa_udf = spark.udf.register('Etapas', edad_etapa)

datos_limpios = datos_limpios.withColumn('Etapa', etapa_udf(datos['Edad']))

datos_limpios.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------+--------------------+----+----+--------------------+---------+------+----------------+----------+---------------------+------------+-----------------+-----------------+-----------------+--------+
|fecha_reporte_web|id_de_caso|fecha_de_notificaci_n|departamento_nom|ciudad_municipio_nom|edad|sexo|fuente_tipo_contagio|ubicacion|estado|pais_viajo_1_nom|recuperado|fecha_inicio_sintomas|fecha_muerte|fecha_diagnostico| fecha_recuperado|tipo_recuperacion|   Etapa|
+-----------------+----------+---------------------+----------------+--------------------+----+----+--------------------+---------+------+----------------+----------+---------------------+------------+-----------------+-----------------+-----------------+--------+
| 6/3/2020 0:00:00|         1|     2/3/2020 0:00:00|          BOGOTA|              BOGOTA|  19|   F|           Importado|     Casa|  Leve|          ITALIA|Recuperado|    27/2/2020 0:00:00|        null| 6/3

In [6]:
datos_limpios = datos_limpios.fillna('COLOMBIA', subset=['pais_viajo_1_nom'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
datos_limpios.select('pais_viajo_1_nom').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|pais_viajo_1_nom|
+----------------+
|          ITALIA|
|          ESPAÑA|
|          ESPAÑA|
|        COLOMBIA|
|        COLOMBIA|
|        COLOMBIA|
|  ESTADOS UNIDOS|
|          ESPAÑA|
|          ESPAÑA|
|          ESPAÑA|
|          ESPAÑA|
|          ITALIA|
|        COLOMBIA|
|          ESPAÑA|
|          ESPAÑA|
|          ITALIA|
|          ESPAÑA|
|          ESPAÑA|
|        COLOMBIA|
|        COLOMBIA|
+----------------+
only showing top 20 rows

In [8]:
datos_limpios.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------+--------------------+----+----+--------------------+---------+------+----------------+----------+---------------------+------------+-----------------+-----------------+-----------------+--------+
|fecha_reporte_web|id_de_caso|fecha_de_notificaci_n|departamento_nom|ciudad_municipio_nom|edad|sexo|fuente_tipo_contagio|ubicacion|estado|pais_viajo_1_nom|recuperado|fecha_inicio_sintomas|fecha_muerte|fecha_diagnostico| fecha_recuperado|tipo_recuperacion|   Etapa|
+-----------------+----------+---------------------+----------------+--------------------+----+----+--------------------+---------+------+----------------+----------+---------------------+------------+-----------------+-----------------+-----------------+--------+
| 6/3/2020 0:00:00|         1|     2/3/2020 0:00:00|          BOGOTA|              BOGOTA|  19|   F|           Importado|     Casa|  Leve|          ITALIA|Recuperado|    27/2/2020 0:00:00|        null| 6/3

In [9]:
from datetime import date

today = date.today()

d4 = today.strftime("%m-%d-%Y")

url = 's3://proyecto-bigdata/curated/{0}'.format(d4)

datos_limpios.coalesce(1).write.format('csv').option('header','True').save(url)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
paises = datos_limpios.groupBy('pais_viajo_1_nom').count().orderBy('count', ascending=False)

paises = paises.withColumnRenamed("pais_viajo_1_nom","pais_donde_viajo")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
paises.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|    pais_donde_viajo|count|
+--------------------+-----+
|            COLOMBIA|  524|
|              ESPAÑA|  191|
|      ESTADOS UNIDOS|  106|
|             ECUADOR|   32|
|              MÉXICO|   22|
|              PANAMA|   15|
|             TURQUÍA|   13|
|REINO UNIDO DE GR...|   12|
|         PUERTO RICO|   10|
|              BRASIL|    9|
|             JAMAICA|    9|
|              ITALIA|    9|
|             FRANCIA|    8|
|ANTILLAS NEERLAND...|    7|
|            ALEMANIA|    5|
|              GRECIA|    4|
|REPÚBLICA DOMINICANA|    3|
|              EGIPTO|    2|
|               CHILE|    2|
|               ARUBA|    2|
+--------------------+-----+
only showing top 20 rows

In [12]:
recuperados_y_fallecidos_departamento = datos_limpios.groupBy('departamento_nom','recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
recuperados_y_fallecidos_departamento.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+-----+
|departamento_nom|recuperado|count|
+----------------+----------+-----+
|          BOGOTA|Recuperado|  379|
|           VALLE|Recuperado|  139|
|       ANTIOQUIA|Recuperado|  105|
|    CUNDINAMARCA|Recuperado|   45|
|       CARTAGENA|Recuperado|   36|
|       RISARALDA|Recuperado|   33|
|    BARRANQUILLA|Recuperado|   29|
|           HUILA|Recuperado|   28|
|         QUINDIO|Recuperado|   23|
|          BOGOTA| Fallecido|   22|
| NORTE SANTANDER|Recuperado|   19|
|           CESAR|Recuperado|   15|
|          CALDAS|Recuperado|   15|
|          TOLIMA|Recuperado|   14|
|       SANTANDER|Recuperado|   12|
|            META|Recuperado|   12|
|           CAUCA|Recuperado|   11|
|  STA MARTA D.E.|Recuperado|   10|
|           VALLE| Fallecido|    8|
|          NARIÑO|Recuperado|    5|
+----------------+----------+-----+
only showing top 20 rows

In [14]:
recuperados_y_fallecidos_etapa = datos_limpios.groupBy('Etapa','recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
recuperados_y_fallecidos_etapa.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+-----+
|           Etapa|recuperado|count|
+----------------+----------+-----+
|         adultez|Recuperado|  629|
|        juventud|Recuperado|  140|
|           vejez|Recuperado|  137|
|           vejez| Fallecido|   30|
|    adolescencia|Recuperado|   28|
|         adultez| Fallecido|   16|
|        infancia|Recuperado|    8|
|primera infancia|Recuperado|    8|
|        juventud| Fallecido|    2|
|         adultez|       N/A|    1|
|           vejez|       N/A|    1|
+----------------+----------+-----+

In [16]:
sexo = datos_limpios.groupBy('sexo').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
sexo.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|sexo|count|
+----+-----+
|   M|  520|
|   F|  480|
+----+-----+

In [18]:
sexo_recuperado = datos_limpios.groupBy('sexo','recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
sexo_recuperado.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+-----+
|sexo|recuperado|count|
+----+----------+-----+
|   M|Recuperado|  488|
|   F|Recuperado|  462|
|   M| Fallecido|   30|
|   F| Fallecido|   18|
|   M|       N/A|    2|
+----+----------+-----+

In [20]:
recuperados_muertos_medellin = datos_limpios.filter(datos_limpios.ciudad_municipio_nom == 'MEDELLIN').groupBy('ciudad_municipio_nom', 'recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
recuperados_muertos_medellin.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-----+
|ciudad_municipio_nom|recuperado|count|
+--------------------+----------+-----+
|            MEDELLIN|Recuperado|   63|
+--------------------+----------+-----+